In [41]:
from src.models.subreddit_analysis import SubredditAnalysis
from imports import *
import plotly.express as px
from nltk.probability import FreqDist


li = ['posts', 'comments', 'results']

In [42]:
pd.set_option('display.max_colwidth', 1000)

# 1. Home - Conflicting Viewpoints in Reddit Submissions

**Welcome to BEReddiT**

***Subreddit Search Bar***  
***Subreddit Submission Bar***

*Brief Subreddit Metadata* [TODO]

- Num of moderators
- Num of followers and members
- Flairs
- Number of posts and comments for analysis

*Basic Subreddit Data*

- Posts
    - Total Number of Posts
- Comments
    - Total Number of Comments

### Number of posts and comments retrieved after preprocessing raw data.

In [43]:
sub = ['antiwork', 'computerscience', 'loseit', 'WorkReform', 'movies', 'geopolitics', 'AmItheAsshole', 'AskReddit'] #dropdown

path = '../data/results/'+sub[-1]+'_hot_'

posts_df = pd.read_csv(path+li[0]+'.csv')
comments_df = pd.read_csv(path+li[1]+'.csv')
res_df = pd.read_csv(path+li[2]+'.csv')

print('posts: ',posts_df.shape)
print('comments: ', comments_df.shape)
print('sub-analysis results: ', res_df.shape)

posts:  (335, 16)
comments:  (9992, 19)
sub-analysis results:  (9997, 35)


In [44]:
posts_df.head()

,index,post_id,title,flair,score,upvote_ratio,subreddit,url,num_comments,body,created,title_word_token,title_tag,body_string,topics,sentiment
0,0,t23tjx,who is the best male character who is not a tough badass?,NaN,2804,0.95,AskReddit,https://www.reddit.com/r/AskReddit/comments/t23tjx/who_is_the_best_male_character_who_isnt_a_tough/,2252,NaN,1.645901e+09,"['best', 'male', 'character', 'tough', 'badass', '?']","[[('best', 'JJS')], [('male', 'NN')], [('character', 'NN')], [('tough', 'JJ')], [('badass', 'NN')], [('?', '.')]]",best male character tough badass ?,1_movie_celebrity_game_character,5
1,1,t1y6g6,what is a fashion trend that you cannot stand?,NaN,4414,0.94,AskReddit,https://www.reddit.com/r/AskReddit/comments/t1y6g6/what_is_a_fashion_trend_that_you_cant_stand/,3840,NaN,1.645885e+09,"['fashion', 'trend', 'stand', '?']","[[('fashion', 'NN')], [('trend', 'NN')], [('stand', 'NN')], [('?', '.')]]",fashion trend stand ?,-1_mask_name_would_people,5
2,2,t1qnsc,what are some common signs that someone grew up with sh*tty parents?,NaN,40599,0.89,AskReddit,https://www.reddit.com/r/AskReddit/comments/t1qnsc/what_are_some_common_signs_that_someone_grew_up/,12133,NaN,1.645858e+09,"['common', 'sign', 'someone', 'grow', 'sh', '*', 'tty', 'parent', '?']","[[('common', 'JJ')], [('signs', 'NNS')], [('someone', 'NN')], [('grew', 'VBD')], [('sh', 'NN')], [('*', 'NN')], [('tty', 'NN')], [('parents', 'NNS')], [('?', '.')]]",common sign someone grow sh * tty parent ?,2_parent_grow_kid_reddit,3
3,3,t1w76y,"men of reddit, what should women know about the penis? [serious]",Serious Replies Only,1528,0.84,AskReddit,https://www.reddit.com/r/AskReddit/comments/t1w76y/men_of_reddit_what_should_women_know_about_the/,1235,NaN,1.645879e+09,"['men', 'reddit', ',', 'woman', 'know', 'penis', '?', '[', 'serious', ']']","[[('men', 'NNS')], [('reddit', 'NN')], [(',', ',')], [('women', 'NNS')], [('know', 'VB')], [('penis', 'NN')], [('?', '.')], [('[', 'NN')], [('serious', 'JJ')], [(']', 'NN')]]","men reddit , woman know penis ? [ serious ]",0_something_people_thing_bad,1
4,4,t1oown,"[serious] girls, what should men know about the vagina?",Serious Replies Only,15379,0.81,AskReddit,https://www.reddit.com/r/AskReddit/comments/t1oown/serious_girls_what_should_men_know_about_the/,5053,NaN,1.645852e+09,"['[', 'serious', ']', 'girl', ',', 'men', 'know', 'vagina', '?']","[[('[', 'NN')], [('serious', 'JJ')], [(']', 'NN')], [('girls', 'NNS')], [(',', ',')], [('men', 'NNS')], [('know', 'VB')], [('vagina', 'NN')], [('?', '.')]]","[ serious ] girl , men know vagina ?",0_something_people_thing_bad,1


In [45]:
comments_df.head()

,index,post_id,comment_id,parent_id,comment,up_vote_count,down_vote_count,controversiality,total_awards_received,score,is_locked,is_collapsed,is_submitter,created_utc,comment_word_token,comment_tag,body_string,topics,sentiment
0,0,t23tjx,hyjs09x,t3_t23tjx,"daniel jackson from stargate sg1. he kind of became more of a badass in the later seasons but originally he was the less physically strong/non-military character. \n\nhis primary strengths were academic accomplishment, cultural/historical expertise, patience and empathy.",644,0,0,1,644,False,False,False,1.645903e+09,"['daniel', 'jackson', 'stargate', 'sg1', '.', 'kind', 'become', 'badass', 'later', 'season', 'originally', 'less', 'physically', 'strong/non-military', 'character', '.', 'primary', 'strength', 'academic', 'accomplishment', ',', 'cultural/historical', 'expertise', ',', 'patience', 'empathy', '.']","[[('daniel', 'NN')], [('jackson', 'NN')], [('stargate', 'NN')], [('sg1', 'NN')], [('.', '.')], [('kind', 'NN')], [('became', 'VBD')], [('badass', 'NN')], [('later', 'RB')], [('seasons', 'NNS')], [('originally', 'RB')], [('less', 'RBR')], [('physically', 'RB')], [('strong/non-military', 'JJ')], [('character', 'NN')], [('.', '.')], [('primary', 'NN')], [('strengths', 'NNS')], [('academic', 'JJ')], [('accomplishment', 'NN')], [(',', ',')], [('cultural/historical', 'JJ')], [('expertise', 'NN')], [(',', ',')], [('patience', 'NN')], [('empathy', 'NN')], [('.', '.')]]","daniel jackson stargate sg1 . kind become badass later season originally less physically strong/non-military character . primary strength academic accomplishment , cultural/historical expertise , patience empathy .",-1_people_get_like_go,4
1,1,t23tjx,hyjmiun,t3_t23tjx,ron stoppable,1394,0,0,1,1394,False,False,False,1.645901e+09,"['ron', 'stoppable']","[[('ron', 'NN')], [('stoppable', 'JJ')]]",ron stoppable,2_john_jim_james_paul,1
2,2,t23tjx,hyjpvft,t3_t23tjx,rincewind the wizzard (he cannot spell) from terry pratchett's discworld series is about as much *not* a badass as it is possible to be.,357,0,0,0,357,False,False,False,1.645902e+09,"['rincewind', 'wizzard', '(', 'spell', ')', 'terry', 'pratchett', ""'s"", 'discworld', 'series', 'much', '*', '*', 'badass', 'possible', '.']","[[('rincewind', 'NN')], [('wizzard', 'NN')], [('(', '(')], [('spell', 'NN')], [(')', ')')], [('terry', 'NN')], [('pratchett', 'NN')], [(""'s"", 'POS')], [('discworld', 'NN')], [('series', 'NN')], [('much', 'JJ')], [('*', 'NN')], [('*', 'NN')], [('badass', 'NN')], [('possible', 'JJ')], [('.', '.')]]",rincewind wizzard ( spell ) terry pratchett 's discworld series much * * badass possible .,-1_people_get_like_go,2
3,3,t23tjx,hyjrrtg,t3_t23tjx,chidi anagonye,2697,0,0,4,2697,False,False,False,1.645903e+09,"['chidi', 'anagonye']","[[('chidi', 'NN')], [('anagonye', 'NN')]]",chidi anagonye,53_kite_joji_kujo_schwnli,1
4,4,t23tjx,hyjpqwb,t3_t23tjx,phil dunphy from modern family\n\nhe makes not being a badass an art form.,1273,0,0,2,1273,False,False,False,1.645902e+09,"['phil', 'dunphy', 'modern', 'family', 'make', 'badass', 'art', 'form', '.']","[[('phil', 'NN')], [('dunphy', 'NN')], [('modern', 'JJ')], [('family', 'NN')], [('makes', 'VBZ')], [('badass', 'NN')], [('art', 'NN')], [('form', 'NN')], [('.', '.')]]",phil dunphy modern family make badass art form .,-1_people_get_like_go,4


In [46]:
posts_df.columns, comments_df.columns, res_df.columns

(Index(['index', 'post_id', 'title', 'flair', 'score', 'upvote_ratio',
        'subreddit', ' url', 'num_comments', 'body', 'created',
        'title_word_token', 'title_tag', 'body_string', 'topics', 'sentiment'],
       dtype='object'),
 Index(['index', 'post_id', 'comment_id', 'parent_id', 'comment',
        'up_vote_count', 'down_vote_count', 'controversiality',
        'total_awards_received', 'score', 'is_locked', 'is_collapsed',
        'is_submitter', 'created_utc', 'comment_word_token', 'comment_tag',
        'body_string', 'topics', 'sentiment'],
       dtype='object'),
 Index(['post_index', 'post_id', 'post_title', 'flair', 'post_score',
        'post_upvote_ratio', 'subreddit', 'post_url', 'num_comments',
        'post_body', 'post_created', 'title_word_token', 'title_tag',
        'post_body_string', 'post_topics', 'post_sentiment', 'comment_index',
        'comment_id', 'parent_id', 'comment', 'comment_up_vote_count',
        'comment_down_vote_count', 'comment_controvers

### Histograms of Score Distribution (Number of upvotes remaining) 

In [47]:
posts_score_hist = px.histogram(posts_df, 
                                x="score", 
                                title='Frequency Distribution of Post Score',
                                labels={'score':'Score', 'count':'Number of Posts'},
                                opacity=0.8,
                                color_discrete_sequence=['indianred'],
                                text_auto=True).update_layout(
                                yaxis_title="Number of Posts")
posts_score_hist.show()

In [48]:
comms_score_hist = px.histogram(comments_df,                                
                                x="score",
                                log_y=True,
                                title='Frequency Distribution of Comment Score',
                                opacity=0.8).update_layout(
                                xaxis_title="Score", yaxis_title="Number of Comments (log scale)")
comms_score_hist.update_xaxes(range=[0, 2000])
comms_score_hist.show()
# notify on app that more can be seen because graph was resized to observe data better.

### Submission Word Counts

In [49]:
comments_df.comment.astype(str)

0       daniel jackson from stargate sg1. he kind of became more of a badass in the later seasons but originally he was the less physically strong/non-military character. \n\nhis primary strengths were academic accomplishment, cultural/historical expertise, patience and empathy.
1                                                                                                                                                                                                                                                                         ron stoppable
2                                                                                                                                              rincewind the wizzard (he cannot spell) from terry pratchett's discworld series is about as much *not* a badass as it is possible to be.
3                                                                                                                                                               

In [51]:
comments_df

,index,post_id,comment_id,parent_id,comment,up_vote_count,down_vote_count,controversiality,total_awards_received,score,is_locked,is_collapsed,is_submitter,created_utc,comment_word_token,comment_tag,body_string,topics,sentiment
0,0,t23tjx,hyjs09x,t3_t23tjx,"daniel jackson from stargate sg1. he kind of became more of a badass in the later seasons but originally he was the less physically strong/non-military character. \n\nhis primary strengths were academic accomplishment, cultural/historical expertise, patience and empathy.",644,0,0,1,644,False,False,False,1.645903e+09,"['daniel', 'jackson', 'stargate', 'sg1', '.', 'kind', 'become', 'badass', 'later', 'season', 'originally', 'less', 'physically', 'strong/non-military', 'character', '.', 'primary', 'strength', 'academic', 'accomplishment', ',', 'cultural/historical', 'expertise', ',', 'patience', 'empathy', '.']","[[('daniel', 'NN')], [('jackson', 'NN')], [('stargate', 'NN')], [('sg1', 'NN')], [('.', '.')], [('kind', 'NN')], [('became', 'VBD')], [('badass', 'NN')], [('later', 'RB')], [('seasons', 'NNS')], [('originally', 'RB')], [('less', 'RBR')], [('physically', 'RB')], [('strong/non-military', 'JJ')], [('character', 'NN')], [('.', '.')], [('primary', 'NN')], [('strengths', 'NNS')], [('academic', 'JJ')], [('accomplishment', 'NN')], [(',', ',')], [('cultural/historical', 'JJ')], [('expertise', 'NN')], [(',', ',')], [('patience', 'NN')], [('empathy', 'NN')], [('.', '.')]]","daniel jackson stargate sg1 . kind become badass later season originally less physically strong/non-military character . primary strength academic accomplishment , cultural/historical expertise , patience empathy .",-1_people_get_like_go,4
1,1,t23tjx,hyjmiun,t3_t23tjx,ron stoppable,1394,0,0,1,1394,False,False,False,1.645901e+09,"['ron', 'stoppable']","[[('ron', 'NN')], [('stoppable', 'JJ')]]",ron stoppable,2_john_jim_james_paul,1
2,2,t23tjx,hyjpvft,t3_t23tjx,rincewind the wizzard (he cannot spell) from terry pratchett's discworld series is about as much *not* a badass as it is possible to be.,357,0,0,0,357,False,False,False,1.645902e+09,"['rincewind', 'wizzard', '(', 'spell', ')', 'terry', 'pratchett', ""'s"", 'discworld', 'series', 'much', '*', '*', 'badass', 'possible', '.']","[[('rincewind', 'NN')], [('wizzard', 'NN')], [('(', '(')], [('spell', 'NN')], [(')', ')')], [('terry', 'NN')], [('pratchett', 'NN')], [(""'s"", 'POS')], [('discworld', 'NN')], [('series', 'NN')], [('much', 'JJ')], [('*', 'NN')], [('*', 'NN')], [('badass', 'NN')], [('possible', 'JJ')], [('.', '.')]]",rincewind wizzard ( spell ) terry pratchett 's discworld series much * * badass possible .,-1_people_get_like_go,2
3,3,t23tjx,hyjrrtg,t3_t23tjx,chidi anagonye,2697,0,0,4,2697,False,False,False,1.645903e+09,"['chidi', 'anagonye']","[[('chidi', 'NN')], [('anagonye', 'NN')]]",chidi anagonye,53_kite_joji_kujo_schwnli,1
4,4,t23tjx,hyjpqwb,t3_t23tjx,phil dunphy from modern family\n\nhe makes not being a badass an art form.,1273,0,0,2,1273,False,False,False,1.645902e+09,"['phil', 'dunphy', 'modern', 'family', 'make', 'badass', 'art', 'form', '.']","[[('phil', 'NN')], [('dunphy', 'NN')], [('modern', 'JJ')], [('family', 'NN')], [('makes', 'VBZ')], [('badass', 'NN')], [('art', 'NN')], [('form', 'NN')], [('.', '.')]]",phil dunphy modern family make badass art form .,-1_people_get_like_go,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9987,9987,t21nmp,hyj7nma,t3_t21nmp,"you just tell her you need those things, so when could you go her place to get them",1,0,0,0,1,False,False,False,1.645895e+09,"['tell', 'need', 'thing', ',', 'could', 'go', 'place', 'get']","[[('tell', 'NN')], [('need', 'NN')], [('things', 'NNS')], [(',', ',')], [('could', 'MD')], [('go', 'VB')], [('place', 'NN')], [('get', 'VB')]]","tell need thing , could go place get",-1_people_get_like_go,5
9988,9988,t21nmp,hyj7y08,t3_t21nmp,i had to break in haha.,1,0,0,0,1,False,False,False,1.645895e+09,"['break', 'haha', '.']","[[('break', 'NN')], [('haha', 'NN'

In [50]:
posts_df['word_counts'] = posts_df.title.str.cat(posts_df.body, sep=" ").str.split().apply(len)
comments_df['word_counts'] = comments_df.comment.astype(str).str.split().apply(len)

TypeError: object of type 'float' has no len()

In [ ]:
res_df[12001]

In [ ]:
fig = px.histogram(posts_df, 
                    x="word_counts",
                    title='Word-Count Distribution for Posts',
                    labels={'word_counts':'Word Count', 'count':'Number of Posts'},
                    opacity=0.8,
                    color_discrete_sequence=['indianred'],
                    text_auto=True).update_layout(
                    yaxis_title="Number of Posts")
fig.show()

In [ ]:
fig = px.histogram(comments_df, 
                   x="word_counts",
                   log_y=True,
                    title='Word-Count Distribution for Comments',
                    opacity=0.8).update_layout(
                    xaxis_title="Word Count", yaxis_title="Number of Comments (log scale)")
comms_score_hist.update_xaxes(range=[0, 2000])
fig.show()

# 2. Relevance - Are the comments in discussions relevant to the submission?

In [ ]:
res_df.comment_relevance

### Frequency Distribution of Overall Comment Relevance

In [ ]:
hist_df = res_df.copy()
hist_df["color"] = np.select(
    [hist_df["comment_relevance"].gt(0.5), hist_df["comment_relevance"].lt(0.5)],
    ["green", "red"],
    "orange"
)
fig = px.histogram(hist_df, x="comment_relevance",
                    title='Distribution of Comment Relevance',
                    labels={'comment_relevance':'Comment Relevance Score', 'count':'Number of Comments'},
                    opacity=0.6, 
                    color="color",
                    color_discrete_map={
                    "green": "green",
                    "red": "red",
                    "orange": "orange"
                    })

fig.update_layout(yaxis_title="Number of Comments", showlegend=False)

fig.add_vline(x=0.5, line_width=3, line_dash="dash", line_color="black")

#include lines for classifying relevant or not : t-test for relevance 0.65< 
fig.show()

In [ ]:
stats.ttest_1samp?

In [ ]:
import scipy.stats as stats

#perform one sample t-test
stats.ttest_1samp(a=res_df.comment_relevance, popmean=0.5, alternative='less')

In [ ]:
np.std(res_df.comment_relevance)

### p-value shows that this sub doesn't significantly better average relevance among its comments

In [ ]:
relevance_mean = np.mean(res_df.comment_relevance)
relevance_mean

### Show Comments after selecting post and Check Relevance in tabular form

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot
init_notebook_mode(connected=True)
prods = list(res_df.head(100).comment)
vals = list(res_df.head(100).comment_relevance)


trace = dict(type = 'table',
             columnwidth= [200] + [100],
             header = dict(height = 50,
                           values = [['<b>Comment</b>'], ['<b>Relevance</b>']],
                           line = dict(color='rgb(50, 50, 50)'),
                           align = 'left', 
                           font = dict(color=['rgb(45, 45, 45)'], size=14),
                           fill = dict( color = 'rgb(235, 235, 235)' )
                              ),
             cells = dict(values = [prods, vals],
                          line = dict(color='#506784'),
                          align = 'left',
                          font = dict(color=['rgb(40, 40, 40)'], size=12),
                          format = [None, ",.2f"],
                          height = 30,
                          fill = dict(color=['rgb(245, 245, 245)',#unique color for the first column
                                            ['rgba(0, 250, 0, 0.8)' if val >= 0.65 else 'rgba(250, 0, 0, 0.8)' for val in vals] ]
                          #the cells in the second column are colored with green or red, according to their values(+ or -)
                                         )
                             )
                  )

In [ ]:
layout = dict(width=1000, autosize=True, 
              title='Cells colored according to their values', showlegend=False)
fig= dict(data=[trace], layout=layout)
iplot(fig)

### Relevance score for a post is the number of relevant comments to a post out of the 100 sampled comments.

In [ ]:
posts_df['post_relevance'] = list(res_df.groupby(["post_id"])['comment_relevance'].mean())
posts_df.post_relevance

### Distribution of relevance scores: the following shows a distribution of how discussion across posts in this subreddit tends to be relevant. (A simpler form of showing overall comment relevance)

In [ ]:
hist_df = posts_df.copy()
hist_df["color"] = np.select(
    [hist_df["post_relevance"].gt(0.5), hist_df["post_relevance"].lt(0.5)],
    ["green", "red"],
    "orange"
)
fig = px.histogram(hist_df, x="post_relevance",
                    title='Distribution of Post Relevance',
                    labels={'post_relevance':'Post Relevance Score', 'count':'Number of Posts'},
                    opacity=0.6, 
                    color="color",
                    color_discrete_map={
                    "green": "green",
                    "red": "red",
                    "orange": "orange"
                    })

fig.update_layout(yaxis_title="Number of Posts", showlegend=False)

fig.add_vline(x=0.5, line_width=3, line_dash="dash", line_color="black")

#include lines for classifying relevant or not : t-test for relevance 0.65< 
fig.show()

# 3. Topic Modeling - What are posts talking about?

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
res_df.comment_topics

In [ ]:
fig = res_df.comment_topics.value_counts()[1:16].plot(kind="bar", title='<b>Top 15</b> Important Topics For Comments',
                    text_auto=True).update_layout(
                    xaxis_title="Topics", yaxis_title="Number of Comments")
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
posts_df.topics.value_counts()

In [ ]:
fig = posts_df.topics.value_counts()[1:].plot(kind="bar", title='Important Topics For Posts',
                    text_auto=True, color_discrete_sequence=['indianred']).update_layout(
                    xaxis_title="Topics", yaxis_title="Number of Posts")
fig.update_layout(showlegend=False)
fig.show()

# 4. Sentiment Analysis - Can we identify conflict with Sentiment?

In [ ]:
hist_df = posts_df.copy()
hist_df["color"] = np.select(
    [hist_df["sentiment"].eq(5), hist_df["sentiment"].eq(1)],
    ["green", "red"],
    "blue"
)

fig = px.histogram(hist_df, 
                   x="sentiment",
                   title='Distribution of Sentiment for Posts',text_auto=True, 
                   color="color",
                   color_discrete_map={
                    "green": "green",
                    "red": "red",
                    "blue": "blue"
                    }).update_layout(
                   xaxis_title="Sentiment", yaxis_title="Number of Posts", showlegend=False)
fig.show()

In [ ]:
hist_df = comments_df.copy()
hist_df["color"] = np.select(
    [hist_df["sentiment"].eq(5), hist_df["sentiment"].eq(1)],
    ["green", "red"],
    "blue"
)

fig = px.histogram(hist_df, 
                   x="sentiment",
                   title='Distribution of Sentiment for Comments',text_auto=True,
                   color="color",
                   color_discrete_map={
                    "green": "green",
                    "red": "red",
                    "blue": "blue"
                    }).update_layout(
                   xaxis_title="Sentiment", yaxis_title="Number of Comments", showlegend=False)
fig.show()

In [ ]:
fig = px.box(posts_df, x="sentiment", y='post_relevance')
fig.show()

In [ ]:
# fig = px.scatter(res_df, x="comment_relevance", y='comment_score', color='sentiment')
# fig.show()

In [ ]:
fig = px.box(res_df, x="post_id", y='comment_relevance')
fig.show()

## Relevance over time